In [1]:
!pip install torch transformers langchain langchain_community faiss-cpu pandas networkx sentencepiece

INFO: pip is looking at multiple versions of langchain-community to determine which version is compatible with other requirements. This could take a while.
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.5/2.5 MB 43.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 31.4/31.4 MB 72.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 64.7/64.7 kB 6.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 50.9/50.9 kB 4.6 MB/s eta 0:00:00
  Attempting uninstall: requests
    Found existing installation: requests 2.32.4
    Uninstalling requests-2.32.4:
      Successfully uninstalled requests-2.32.4
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
google-colab 1.0.0 requires requests==2.32.4, but you have requests 2.32.5 which is incompatible.


In [45]:
import os
import asyncio, random, uuid, json
from datetime import datetime, timedelta, timezone
import pandas as pd
import networkx as nx

from transformers import AutoModelForCausalLM, AutoTokenizer, pipeline
from langchain.llms import HuggingFacePipeline
from langchain.prompts import PromptTemplate
from langchain.vectorstores import FAISS
from langchain.docstore.document import Document
from langchain.chains import RetrievalQA
from langchain.embeddings import HuggingFaceEmbeddings

# ---------------- Configuration ----------------
STREAM_RATE_PER_SEC = 5           # events/sec
WINDOW_SECONDS = 60               # sliding window
BLOCK_CONFIDENCE_THRESHOLD = 0.8

In [46]:
profiles = {
    "user_1": {"country":"IN","risk_score":0.2,"devices":["dev_1"]},
    "user_2": {"country":"US","risk_score":0.7,"devices":["dev_2","dev_3"]},
    "user_3": {"country":"FR","risk_score":0.1,"devices":["dev_4"]},
}

# Static blacklist initially
blacklist = pd.DataFrame([
    {"entity_type":"ip","entity_id":"203.0.113.5","reason":"chargeback_ring"},
    {"entity_type":"card","entity_id":"4111111111111111","reason":"fraud_history"},
    {"entity_type":"device","entity_id":"dev_9","reason":"stolen"}
])

In [47]:
async def transaction_stream(queue):
    users = list(profiles.keys())
    while True:
        uid = random.choice(users)
        txn = {
            "txn_id": str(uuid.uuid4()),
            "user_id": uid,
            "device": random.choice(profiles[uid]["devices"]),
            "amount": round(random.uniform(10, 5000), 2),
            "ip": random.choice(["198.51.100.1", "203.0.113.5", "192.0.2.25"]),
            "timestamp": datetime.now(timezone.utc)
        }
        await queue.put(txn)
        await asyncio.sleep(1 / STREAM_RATE_PER_SEC)

In [48]:
class FraudProcessor:
    def __init__(self):
        self.df = pd.DataFrame(columns=["user_id", "amount", "timestamp"])
        self.graph = nx.Graph()

    def enrich(self, txn):
        prof = profiles.get(txn["user_id"], {})
        txn.update({
            "country": prof.get("country", "UNK"),
            "risk_score": prof.get("risk_score", 0.5)
        })
        txn["blacklisted"] = (
            (blacklist["entity_id"] == txn["ip"]).any() or
            (blacklist["entity_id"] == txn["device"]).any()
        )
        return txn

    def update_state(self, txn):
        ts = txn["timestamp"]
        if ts.tzinfo is None:
            ts = ts.replace(tzinfo=timezone.utc)
        self.df.loc[len(self.df)] = [txn["user_id"], txn["amount"], ts]

        # Graph link (user ↔ IP)
        self.graph.add_edge(txn["user_id"], txn["ip"])

        # Sliding window
        window_start = datetime.now(timezone.utc) - timedelta(seconds=WINDOW_SECONDS)
        self.df = self.df[self.df["timestamp"] > window_start]

    def compute_features(self, txn):
        df_user = self.df[self.df["user_id"] == txn["user_id"]]
        txn["avg_amount_1m"] = df_user["amount"].mean() if len(df_user) > 0 else 0
        txn["velocity_1m"] = len(df_user)
        return txn

    def anomaly_score(self, txn):
        mu = self.df["amount"].mean() if len(self.df) > 5 else 0
        sigma = self.df["amount"].std() if len(self.df) > 5 else 1
        z = abs((txn["amount"] - mu) / (sigma + 1e-6))
        return min(z / 10, 1.0)

In [49]:
async def send_alert(txn, reason):
    print(f"🚨 ALERT [{txn['txn_id'][:8]}] user={txn['user_id']} reason={reason} amt={txn['amount']}")

def draft_SAR(txn, reason, proc):
    return (
        f"SAR Draft:\n"
        f"User {txn['user_id']} triggered alert '{reason}'. "
        f"Amount={txn['amount']}, Country={txn['country']}, "
        f"Velocity={txn.get('velocity_1m')}. "
        f"Graph degree={len(list(proc.graph.neighbors(txn['user_id'])))}."
    )

def update_blacklist(txn, reason):
    global blacklist
    if txn["ip"] not in blacklist["entity_id"].values:
        blacklist = pd.concat([blacklist, pd.DataFrame([{
            "entity_type": "ip",
            "entity_id": txn["ip"],
            "reason": reason
        }])], ignore_index=True)
    if txn["device"] not in blacklist["entity_id"].values:
        blacklist = pd.concat([blacklist, pd.DataFrame([{
            "entity_type": "device",
            "entity_id": txn["device"],
            "reason": reason
        }])], ignore_index=True)

In [8]:
model_name = "TheBloke/vicuna-7B-1.1-HF"
tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModelForCausalLM.from_pretrained(model_name, device_map="auto", torch_dtype="auto")

pipe = pipeline(
    "text-generation",
    model=model,
    tokenizer=tokenizer,
    max_new_tokens=256,
    temperature=0.3,
    repetition_penalty=1.1
)

llm = HuggingFacePipeline(pipeline=pipe)

/usr/local/lib/python3.12/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/727 [00:00<?, ?B/s]

tokenizer.model:   0%|          | 0.00/500k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/411 [00:00<?, ?B/s]

You are using the default legacy behaviour of the <class 'transformers.models.llama.tokenization_llama.LlamaTokenizer'>. This is expected, and simply means that the `legacy` (previous) behavior will be used so nothing changes for you. If you want to use the new behaviour, set `legacy=False`. This should only be set if you understand what it means, and thoroughly read the reason why this was added as explained in https://github.com/huggingface/transformers/pull/24565 - if you loaded a llama tokenizer from a GGUF file you can ignore this message
You are using the default legacy behaviour of the <class 'transformers.models.llama.tokenization_llama_fast.LlamaTokenizerFast'>. This is expected, and simply means that the `legacy` (previous) behavior will be used so nothing changes for you. If you want to use the new behaviour, set `legacy=False`. This should only be set if you understand what it means, and thoroughly read the reason why this was added as explained in https://github.com/huggin

...

pytorch_model-00001-of-00002.bin:   0%|          | 0.00/9.98G [00:00<?, ?B/s]

The following generation flags are not valid and may be ignored: ['pad_token_id']. Set `TRANSFORMERS_VERBOSITY=info` for more details.


Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

generation_config.json:   0%|          | 0.00/137 [00:00<?, ?B/s]

Device set to use cuda:0
/tmp/ipython-input-1455396502.py:14: LangChainDeprecationWarning: The class `HuggingFacePipeline` was deprecated in LangChain 0.0.37 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-huggingface package and should be used instead. To use it run `pip install -U :class:`~langchain-huggingface` and import as `from :class:`~langchain_huggingface import HuggingFacePipeline``.
  llm = HuggingFacePipeline(pipeline=pipe)


In [50]:

# Fraud knowledge documents
docs = [
    Document(page_content="Any transaction from blacklisted IPs should be blocked immediately."),
    Document(page_content="High velocity transactions in 1 minute window may indicate fraud."),
    Document(page_content="Transactions above $4000 require additional verification."),
]

# FAISS vectorstore
embed_model = HuggingFaceEmbeddings(model_name="sentence-transformers/all-MiniLM-L6-v2")
vectorstore = FAISS.from_documents(docs, embed_model)

# RAG chain
rag_chain = RetrievalQA.from_chain_type(
    llm=llm,
    chain_type="stuff",
    retriever=vectorstore.as_retriever(),
    return_source_documents=True
)

In [53]:
def generate_sar_with_rag(txn, reason, rag_chain):
    txn_str = json.dumps(txn, default=str)
    query_text = f"Explain why this transaction is suspicious: {txn_str} Reason: {reason}"

    # ✅ Make sure the key matches what RetrievalQA expects
    outputs = rag_chain.invoke({"query": query_text})

    # Print the generated explanation
    print(f"📝 RAG SAR Analysis:\n{outputs['result']}\n{'-'*50}")

In [54]:
proc = FraudProcessor()

async def main():
    queue = asyncio.Queue()
    asyncio.create_task(transaction_stream(queue))
    print("✅ Fraud Monitor running...")

    while True:
        txn = await queue.get()
        txn = proc.enrich(txn)
        proc.update_state(txn)
        txn = proc.compute_features(txn)
        score = proc.anomaly_score(txn)

        if score > BLOCK_CONFIDENCE_THRESHOLD or txn["blacklisted"]:
            reason = "High anomaly" if not txn["blacklisted"] else "Blacklist hit"

            await send_alert(txn, reason)

            # Update blacklist dynamically
            update_blacklist(txn, reason)

            # Draft SAR
            sar_text = draft_SAR(txn, reason, proc)
            print(sar_text, "\n" + "-"*50)

            # RAG-based SAR
            generate_sar_with_rag(txn, reason, rag_chain)

await main()

You seem to be using the pipelines sequentially on GPU. In order to maximize efficiency please use a dataset


✅ Fraud Monitor running...
🚨 ALERT [e23d9e82] user=user_1 reason=Blacklist hit amt=233.81
SAR Draft:
User user_1 triggered alert 'Blacklist hit'. Amount=233.81, Country=IN, Velocity=1. Graph degree=1. 
--------------------------------------------------
📝 RAG SAR Analysis:
Use the following pieces of context to answer the question at the end. If you don't know the answer, just say that you don't know, don't try to make up an answer.

Any transaction from blacklisted IPs should be blocked immediately.

High velocity transactions in 1 minute window may indicate fraud.

Transactions above $4000 require additional verification.

Question: Explain why this transaction is suspicious: {"txn_id": "e23d9e82-4cd9-4d89-ab57-55e37d591012", "user_id": "user_1", "device": "dev_1", "amount": 233.81, "ip": "198.51.100.1", "timestamp": "2025-10-22 23:03:09.609431+00:00", "country": "IN", "risk_score": 0.2, "blacklisted": "True", "avg_amount_1m": 233.81, "velocity_1m": 1} Reason: Blacklist hit
Helpful An

CancelledError: 

In [44]:
await main()

✅ Fraud Monitor running...
🚨 ALERT [d876c6f1] user=user_2 reason=Blacklist hit amt=1329.01
SAR Draft:
User user_2 triggered alert 'Blacklist hit'. Amount=1329.01, Country=US, Velocity=1. Graph degree=3. 
--------------------------------------------------
📝 RAG SAR Analysis:
Use the following pieces of context to answer the question at the end. If you don't know the answer, just say that you don't know, don't try to make up an answer.

Any transaction from blacklisted IPs should be blocked immediately.

High velocity transactions in 1 minute window may indicate fraud.

Transactions above $4000 require additional verification.

Question: Explain why this transaction is suspicious: {"txn_id": "d876c6f1-4be6-420b-a2b9-8c32ea783281", "user_id": "user_2", "device": "dev_2", "amount": 1329.01, "ip": "203.0.113.5", "timestamp": "2025-10-22 22:51:31.106271+00:00", "country": "US", "risk_score": 0.7, "blacklisted": "True", "avg_amount_1m": 1329.01, "velocity_1m": 1} Reason: Blacklist hit
Helpful

CancelledError: 